## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/123-googleq-a-train-3fold-bert-base-unc-categoryv4/' + '*.h5')
model_path_list.sort()
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/123-googleq-a-train-3fold-bert-base-unc-categoryv4/model_fold_1.h5', '/kaggle/input/123-googleq-a-train-3fold-bert-base-unc-categoryv4/model_fold_2.h5', '/kaggle/input/123-googleq-a-train-3fold-bert-base-unc-categoryv4/model_fold_3.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512
N_CLASS_CAT = test['category'].nunique()

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output_seq = Dense(N_CLASS, activation="sigmoid", name="output_seq")(x)
    output_class = Dense(N_CLASS_CAT, activation="softmax", name="output_class")(pooled_output)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=[output_seq, output_class])
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test)[0][: ,:N_CLASS] / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.933550,0.660419,0.145719,0.469545,0.710778,0.601613,0.680534,0.618385,0.502064,...,0.911417,0.914681,0.566143,0.965111,0.956627,0.803501,0.121235,0.063140,0.788101,0.917057
1,46,0.889088,0.489631,0.005218,0.753832,0.786928,0.931911,0.524984,0.405199,0.051552,...,0.693502,0.941923,0.677711,0.964079,0.979710,0.863180,0.922944,0.111012,0.123254,0.884260
2,70,0.919427,0.679232,0.019305,0.804137,0.888172,0.943838,0.582898,0.525862,0.092549,...,0.865690,0.937900,0.641095,0.974377,0.972881,0.898062,0.191757,0.066346,0.820998,0.929904
3,132,0.865561,0.430787,0.008661,0.631705,0.758267,0.897590,0.507356,0.450012,0.145947,...,0.727321,0.935499,0.645942,0.961299,0.978351,0.882562,0.880343,0.186050,0.283355,0.900063
4,200,0.892579,0.463224,0.027478,0.889640,0.812981,0.880713,0.591266,0.603531,0.086844,...,0.669686,0.912555,0.660222,0.954705,0.956162,0.855373,0.135480,0.069934,0.615025,0.922138


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.887433,0.579137,0.030016,0.696636,0.798609,0.846925,0.573162,0.482592,0.225048,...,0.791359,0.929548,0.663723,0.959796,0.970392,0.859297,0.565055,0.123442,0.471098,0.911822
std,2812.670060,0.044270,0.126965,0.051577,0.108616,0.103423,0.127410,0.050636,0.081008,0.192678,...,0.081133,0.020316,0.042286,0.010426,0.010777,0.033179,0.319023,0.058726,0.262519,0.017274
min,39.000000,0.715852,0.333737,0.003007,0.204702,0.335018,0.208512,0.455247,0.349634,0.006598,...,0.578704,0.849207,0.523336,0.913447,0.921730,0.747400,0.005178,0.007985,0.051491,0.855424
25%,2572.000000,0.857085,0.470313,0.007300,0.636016,0.744158,0.825663,0.534774,0.424239,0.080225,...,0.727826,0.918209,0.635203,0.953710,0.964209,0.838165,0.239780,0.074082,0.240170,0.900767
50%,5093.000000,0.887267,0.561577,0.011138,0.695478,0.806023,0.892471,0.566351,0.456410,0.154285,...,0.792704,0.932628,0.664975,0.961119,0.971846,0.862575,0.658365,0.127585,0.421550,0.912511
75%,7482.000000,0.922456,0.681640,0.024023,0.761083,0.873341,0.924653,0.604652,0.521484,0.312867,...,0.864024,0.945305,0.691239,0.967070,0.978701,0.883587,0.857179,0.164994,0.688857,0.924614
max,9640.000000,0.974579,0.880795,0.448949,0.978380,0.977234,0.987353,0.713850,0.728938,0.851894,...,0.959953,0.976750,0.799886,0.981939,0.988876,0.945550,0.955424,0.294486,0.981857,0.953578
